# Full Adder
Below is a Logisim diagram a Full Adder circuit implementation. 
![](images/full_adder_logisim.png)

In [1]:
from magma import *

![](images/SB_LUT4_diagram.png)
![](images/SB_LUT4_verilog.png)

In [2]:
SB_LUT4 = DeclareCircuit('SB_LUT4',
                         "I0", In(Bit),
                         "I1", In(Bit),
                         "I2", In(Bit),
                         "I3", In(Bit),
                         "O",  Out(Bit))

A0 = 0xAAAA
A1 = 0xCCCC
A2 = 0xF0F0
A3 = 0xFF00

In [3]:
class And2(Circuit):
    name = "And2"
    IO = ["I0", In(Bit), "I1", In(Bit), "O", Out(Bit)]
    
    @classmethod
    def definition(io):
        lut = SB_LUT4(LUT_INIT=(A0&A1, 16))
        wire(io.I0, lut.I0)
        wire(io.I1, lut.I1)
        wire(0, lut.I2)
        wire(0, lut.I3)
        wire(lut.O, io.O)

class Or3(Circuit):
    name = "Or3"
    IO = ["I0", In(Bit), "I1", In(Bit), "I2", In(Bit), "O", Out(Bit)]
    
    @classmethod
    def definition(io):
        lut = SB_LUT4(LUT_INIT=(A0|A1|A3, 16))
        wire(io.I0, lut.I0)
        wire(io.I1, lut.I1)
        wire(io.I2, lut.I2)
        wire(0, lut.I3)
        wire(lut.O, io.O)


class Xor3(Circuit):
    name = "Xor3"
    IO = ["I0", In(Bit), "I1", In(Bit), "I2", In(Bit), "O", Out(Bit)]
    
    @classmethod
    def definition(io):
        lut = SB_LUT4(LUT_INIT=(A0^A1^A3, 16))
        wire(io.I0, lut.I0)
        wire(io.I1, lut.I1)
        wire(io.I2, lut.I2)
        wire(0, lut.I3)
        wire(lut.O, io.O)

In [4]:
def and2(a, b):
    return And2()(a, b)

def or3(a, b, c):
    return Or3()(a, b, c)

def xor3(a, b, c):
    return Xor3()(a, b, c)

In [5]:
class FullAdder(Circuit):
    name = "FullAdder"
    IO = ["a", In(Bit), "b", In(Bit), "cin", In(Bit), 
          "out", Out(Bit), "cout", Out(Bit)]
    @classmethod
    def definition(io):
        # Generate the sum
        out = xor3(io.a, io.b, io.cin)
        wire(out, io.out)
        # Generate the carry
        a_and_b = and2(io.a, io.b)
        b_and_cin = and2(io.b, io.cin)
        a_and_cin = and2(io.a, io.cin)
        cout = or3(a_and_b, b_and_cin, a_and_cin)
        wire(cout, io.cout)


In [6]:
from magma.backend.verilog import compile as compile_verilog

print(compile_verilog(FullAdder))

compiling Xor3
compiling And2
compiling Or3
compiling FullAdder
module Xor3 (input  I0, input  I1, input  I2, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h9966)) inst0 (.I0(I0), .I1(I1), .I2(I2), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module And2 (input  I0, input  I1, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'h8888)) inst0 (.I0(I0), .I1(I1), .I2(1'b0), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module Or3 (input  I0, input  I1, input  I2, output  O);
wire  inst0_O;
SB_LUT4 #(.LUT_INIT(16'hFFEE)) inst0 (.I0(I0), .I1(I1), .I2(I2), .I3(1'b0), .O(inst0_O));
assign O = inst0_O;
endmodule

module FullAdder (input  a, input  b, input  cin, output  out, output  cout);
wire  inst0_O;
wire  inst1_O;
wire  inst2_O;
wire  inst3_O;
wire  inst4_O;
Xor3 inst0 (.I0(a), .I1(b), .I2(cin), .O(inst0_O));
And2 inst1 (.I0(a), .I1(b), .O(inst1_O));
And2 inst2 (.I0(b), .I1(cin), .O(inst2_O));
And2 inst3 (.I0(a), .I1(cin), .O(inst3_O));
Or3 inst4 (.I0(inst1_O), .I1(

In [7]:
from loam.boards.icestick import IceStick
icestick = IceStick()

icestick.Clock.on()
icestick.D1.on()
icestick.D2.on()

main = icestick.main()
adder = FullAdder()
wire(0, adder.a)
wire(1, adder.b)
wire(1, adder.cin)
wire(adder.out, main.D1)
wire(adder.cout, main.D2)

import mantle lattice ice40
import mantle lattice mantle40


In [8]:
compile("build/ice_full_adder", main)

compiling Xor3
compiling And2
compiling Or3
compiling FullAdder
compiling main


In [9]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif ice_full_adder.blif' ice_full_adder.v
arachne-pnr -q -d 1k -o ice_full_adder.txt -p ice_full_adder.pcf ice_full_adder.blif
icepack ice_full_adder.txt ice_full_adder.bin

iceprog ice_full_adder.bin

init..
cdone: high
reset..
cdone: low
flash ID: 0x20 0xBA 0x16 0x10 0x00 0x00 0x23 0x51 0x73 0x10 0x23 0x00 0x35 0x00 0x35 0x06 0x06 0x15 0x43 0xB6
file size: 32220
erase 64kB sector at 0x000000..
programming..
reading..
VERIFY OK
cdone: high
Bye.
